## Setup

In [4]:
!pip install tensorflow-addons


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 9.8 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import pandas as pd
import os
import io
import time


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [7]:
class DataProcessing:
    def __init__(self):
        self.inp_lang_tokenizer = None
        self.targ_lang_tokenizer = None

    ## Step 1 and Step 2
    def sentence_processing(self, sentence):
        sentence = sentence.lower().strip()
        sentence = self.unicode_to_ascii(sentence)

        sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
        sentence = re.sub(r'[" "]+', " ", sentence)
        sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
        sentence = sentence.strip()

        #SOS = <, EOS = >
        sentence = '<' + sentence + '>'
        return sentence

    def unicode_to_ascii(self, s):
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

    def tokenize(self, inp_lang, targ_lang):
        lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='#',char_level=True)
        lang_tokenizer.fit_on_texts(targ_lang)

        tensor = lang_tokenizer.texts_to_sequences(inp_lang)
        tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

        tensor2 = lang_tokenizer.texts_to_sequences(targ_lang)
        tensor2 = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

        return tensor, lang_tokenizer, tensor2, lang_tokenizer

    def load_dataset(self, num_examples=None):
        df = pd.read_csv("Misspelling_Corpus.csv")
        df['Original'] = df['Original'].apply(self.sentence_processing)
        df['Misspelt'] = df['Misspelt'].apply(self.sentence_processing)

        targ_lang = df['Original'].values


        inp_lang = column_array = df['Misspelt'].values


        input_tensor, inp_lang_tokenizer,target_tensor, targ_lang_tokenizer = self.tokenize(inp_lang,targ_lang)

        return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

    def call(self, num_examples, BUFFER_SIZE, BATCH_SIZE):

        input_tensor, target_tensor, self.inp_lang_tokenizer, self.targ_lang_tokenizer = self.load_dataset(num_examples)

        input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

        train_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train))
        train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

        val_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val))
        val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

        return train_dataset, val_dataset, self.inp_lang_tokenizer, self.targ_lang_tokenizer

In [8]:
BUFFER_SIZE = 100000
BATCH_SIZE = 128
# Let's limit the #training examples for faster training
num_examples = 100000

dataset_creator = DataProcessing()
train_dataset, val_dataset, inp_lang, targ_lang = dataset_creator.call(num_examples, BUFFER_SIZE, BATCH_SIZE)

In [9]:
for word, index in inp_lang.word_index.items():
    print(f"Word: {word}, Index: {index}")


Word: #, Index: 1
Word:  , Index: 2
Word: a, Index: 3
Word: e, Index: 4
Word: i, Index: 5
Word: s, Index: 6
Word: o, Index: 7
Word: r, Index: 8
Word: t, Index: 9
Word: n, Index: 10
Word: l, Index: 11
Word: d, Index: 12
Word: m, Index: 13
Word: c, Index: 14
Word: h, Index: 15
Word: b, Index: 16
Word: <, Index: 17
Word: >, Index: 18
Word: p, Index: 19
Word: u, Index: 20
Word: ., Index: 21
Word: v, Index: 22
Word: g, Index: 23
Word: f, Index: 24
Word: y, Index: 25
Word: w, Index: 26
Word: k, Index: 27
Word: ,, Index: 28
Word: x, Index: 29
Word: j, Index: 30
Word: q, Index: 31
Word: z, Index: 32


In [10]:
example_input_batch, example_target_batch = next(iter(train_dataset))
example_input_texts = [inp_lang.sequences_to_texts([sequence.numpy()])[0] for sequence in example_input_batch]
example_input_batch.shape, example_target_batch.shape, example_input_texts


(TensorShape([128, 109]),
 TensorShape([128, 109]),
 ['< a b e l i a n   g o o p e s   a r e f   g r o u p s   . > # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #',
  '< a b d o m e n s   h a s e   p a r d   l o b e s   . > # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #',
  '< a b a   i s   b a s e d   o n   t u   s c i e n c e   o f   p h y s i c o l o g y   . > # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #',
  '< m o s y   a a r d v a r k s   h a a v   w e i g h t   . > # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #',
  '< a a r d v a r k s   a m e   p a r t   b r e a s t s

### Some important parameters

In [11]:
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1
max_length_input = example_input_batch.shape[1]
max_length_output = example_target_batch.shape[1]

embedding_dimentions = 256
units = 1024
steps_per_epoch = num_examples//BATCH_SIZE


In [12]:
print("input len, output len, vocab size")
max_length_input, max_length_output, vocab_inp_size

input len, output len, vocab size


(109, 109, 33)

In [13]:
#####

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz, encoder_cell = "LSTM"):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)


    if encoder_cell == "RNN":
        self.encoder_layer = tf.keras.layers.SimpleRNN(self.enc_units,
                                                      return_sequences=True,
                                                      return_state=True,
                                                      recurrent_initializer='glorot_uniform')
    elif encoder_cell == "GRU":
        self.encoder_layer = tf.keras.layers.GRU(self.enc_units,
                                                  return_sequences=True,
                                                  return_state=True,
                                                  recurrent_initializer='glorot_uniform')
    else:
      self.lstm_layer = tf.keras.layers.LSTM(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')



  def call(self, x, hidden):
    x = self.embedding(x)
    output, h, c = self.lstm_layer(x, initial_state = hidden)
    return output, h, c

  def initialize_hidden_state(self):
    return [tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))]

In [14]:
## Test Encoder Stack

encoder = Encoder(vocab_inp_size, embedding_dimentions, units, BATCH_SIZE)


# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_h, sample_c = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder h vecotr shape: (batch size, units) {}'.format(sample_h.shape))
print ('Encoder c vector shape: (batch size, units) {}'.format(sample_c.shape))

Encoder output shape: (batch size, sequence length, units) (128, 109, 1024)
Encoder h vecotr shape: (batch size, units) (128, 1024)
Encoder c vector shape: (batch size, units) (128, 1024)


In [15]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, attention_type='luong'):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.attention_type = attention_type

    # Embedding Layer
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.fc = tf.keras.layers.Dense(vocab_size)
    self.decoder_rnn_cell = tf.keras.layers.LSTMCell(self.dec_units)
    self.sampler = tfa.seq2seq.sampler.TrainingSampler()

    # Create attention mechanism with memory = None
    mem_seq_len = self.batch_sz*[max_length_input]

    if(self.attention_type=='bahdanau'):
      # modified paper
      self.attention_mechanism = tfa.seq2seq.BahdanauAttention(units=self.dec_units, memory=None, memory_sequence_length=mem_seq_len)
    else:
      #original paper
      self.attention_mechanism = tfa.seq2seq.LuongAttention(units=self.dec_units, memory=None, memory_sequence_length=mem_seq_len)


    self.rnn_cell = self.build_rnn_cell(batch_sz)

    # Define the decoder with respect to fundamental rnn cell
    self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)


  def build_rnn_cell(self, batch_sz):
    rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnn_cell,
                                  self.attention_mechanism, attention_layer_size=self.dec_units)
    return rnn_cell

  def build_initial_state(self, batch_sz, encoder_state, Dtype):
    decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=batch_sz, dtype=Dtype)
    decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
    return decoder_initial_state


  def call(self, inputs, initial_state):
    x = self.embedding(inputs)
    outputs, _, _ = self.decoder(x, initial_state=initial_state, sequence_length=self.batch_sz*[max_length_output-1])
    return outputs


In [16]:
# Test decoder stack

decoder = Decoder(vocab_tar_size, embedding_dimentions, units, BATCH_SIZE, 'luong')
sample_x = tf.random.uniform((BATCH_SIZE, max_length_output))
decoder.attention_mechanism.setup_memory(sample_output)
initial_state = decoder.build_initial_state(BATCH_SIZE, [sample_h, sample_c], tf.float32)


sample_decoder_outputs = decoder(sample_x, initial_state)

print("Decoder Outputs Shape: ", sample_decoder_outputs.rnn_output.shape)


Decoder Outputs Shape:  (128, 108, 33)


## Define the optimizer and the loss function

In [17]:
optimizer = tf.keras.optimizers.Adam()


def loss_function(real, pred):
  cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
  loss = cross_entropy(y_true=real, y_pred=pred)

  # calculate mask loss accuracy
  mask = tf.logical_not(tf.math.equal(real,0))
  mask = tf.cast(mask, dtype=loss.dtype)

  loss = mask* loss
  loss = tf.reduce_mean(loss)
  return loss


def masked_accuracy(real, pred):
    # Ignore padding tokens
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    mask = tf.cast(mask, dtype=tf.float32)
    correct_predictions = tf.cast(tf.equal(real, tf.argmax(pred, axis=-1, output_type=tf.int32)), dtype=tf.float32)
    correct_predictions *= mask
    accuracy = tf.reduce_sum(correct_predictions) / tf.maximum(tf.reduce_sum(mask), 1)

    return accuracy


## Checkpoints (Object-based saving)

In [18]:
checkpoint_dir = '/content/gdrive/MyDrive/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## One train_step operations

In [19]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    acc = 0

    with tf.GradientTape() as tape:
        enc_output, enc_h, enc_c = encoder(inp, enc_hidden)

        # Ignore SOS and EOS tokens (<,>)
        dec_input = targ[:, :-1]
        real = targ[:, 1:]

        decoder.attention_mechanism.setup_memory(enc_output)
        decoder_initial_state = decoder.build_initial_state(BATCH_SIZE, [enc_h, enc_c], tf.float32)
        pred = decoder(dec_input, decoder_initial_state)
        logits = pred.rnn_output
        loss = loss_function(real, logits)
        acc = masked_accuracy(real, logits)

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss, acc


In [20]:
@tf.function
def validation_step(inp, targ, enc_hidden):
    loss = 0
    acc = 0

    enc_output, enc_h, enc_c = encoder(inp, enc_hidden)

    dec_input = targ[:, :-1]
    real = targ[:, 1:]

    decoder.attention_mechanism.setup_memory(enc_output)
    decoder_initial_state = decoder.build_initial_state(BATCH_SIZE, [enc_h, enc_c], tf.float32)
    pred = decoder(dec_input, decoder_initial_state)
    logits = pred.rnn_output
    loss = loss_function(real, logits)
    acc = masked_accuracy(real, logits)

    return loss, acc


## Train the model

In [21]:
EPOCHS = 10

# Lists to store additional variables
losses = []
accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    total_accuracy = 0

    for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
        batch_loss, batch_acc = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        total_accuracy += batch_acc

        # Store batch losses and accuracies
        losses.append(batch_loss.numpy())
        accuracies.append(batch_acc.numpy())

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1,
                                                                         batch,
                                                                         batch_loss.numpy(),
                                                                         batch_acc.numpy()))

    # Validation pass
    val_total_loss = 0
    val_total_accuracy = 0
    num_val_batches = 0

    for val_inp, val_targ in val_dataset:
        val_batch_loss, val_batch_acc = validation_step(val_inp, val_targ, enc_hidden)
        val_total_loss += val_batch_loss
        val_total_accuracy += val_batch_acc
        num_val_batches += 1

    val_loss = val_total_loss / num_val_batches
    val_accuracy = val_total_accuracy / num_val_batches

    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    # Saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix=checkpoint_prefix)

    print('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1,
                                                         total_loss / steps_per_epoch,
                                                         total_accuracy / steps_per_epoch))
    print('Validation Loss {:.4f}, Validation Accuracy {:.4f}'.format(val_loss, val_accuracy))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

# After training, you can process and analyze the recorded metrics as needed
# For example, print the losses, accuracies, validation losses, and validation accuracies
print("Train Losses:", losses)
print("Train Accuracies:", accuracies)
print("Validation Losses:", val_losses)
print("Validation Accuracies:", val_accuracies)


Epoch 1 Batch 0 Loss 1.9315 Accuracy 0.0319
Epoch 1 Batch 100 Loss 1.1168 Accuracy 0.3878
Epoch 1 Batch 200 Loss 0.8255 Accuracy 0.5204
Epoch 1 Batch 300 Loss 0.5785 Accuracy 0.6579
Epoch 1 Batch 400 Loss 0.3826 Accuracy 0.7997
Epoch 1 Batch 500 Loss 0.2535 Accuracy 0.8810
Epoch 1 Batch 600 Loss 0.2054 Accuracy 0.9011
Epoch 1 Loss 0.5168 Accuracy 0.5140
Validation Loss 0.2064, Validation Accuracy 0.9036
Time taken for 1 epoch 573.3168127536774 sec

Epoch 2 Batch 0 Loss 0.2136 Accuracy 0.9069
Epoch 2 Batch 100 Loss 0.1927 Accuracy 0.9165
Epoch 2 Batch 200 Loss 0.1572 Accuracy 0.9298
Epoch 2 Batch 300 Loss 0.1607 Accuracy 0.9229
Epoch 2 Batch 400 Loss 0.1522 Accuracy 0.9308
Epoch 2 Batch 500 Loss 0.1328 Accuracy 0.9327
Epoch 2 Batch 600 Loss 0.1450 Accuracy 0.9317
Epoch 2 Loss 0.1276 Accuracy 0.7408
Validation Loss 0.1367, Validation Accuracy 0.9370
Time taken for 1 epoch 564.8701152801514 sec

Epoch 3 Batch 0 Loss 0.1418 Accuracy 0.9305
Epoch 3 Batch 100 Loss 0.1315 Accuracy 0.9403
Epoc

KeyboardInterrupt: 

## Use tf-addons BasicDecoder for decoding


In [22]:
def evaluate_sentence(sentence):
    sentence = dataset_creator.sentence_processing(sentence)

    # Tokenize input sentence at the character level
    inputs = [inp_lang.word_index[char] for char in sentence]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_input,
                                                          padding='post')
    inputs = tf.convert_to_tensor(inputs)
    inference_batch_size = inputs.shape[0]
    result = ''

    enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size, units))]
    enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)

    dec_h = enc_h
    dec_c = enc_c

    start_tokens = tf.fill([inference_batch_size], targ_lang.word_index['<'])
    end_token = targ_lang.word_index['>']

    greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()

    # Instantiate BasicDecoder object
    decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.rnn_cell, sampler=greedy_sampler, output_layer=decoder.fc)

    # Setup Memory and inital state in decoder
    decoder.attention_mechanism.setup_memory(enc_out)
    decoder_initial_state = decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)
    decoder_embedding_matrix = decoder.embedding.variables[0]

    # decode
    outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens=start_tokens, end_token=end_token, initial_state=decoder_initial_state)
    return outputs.sample_id.numpy()

def spellcheck(sentence):
    result = evaluate_sentence(sentence)
    result_text = targ_lang.sequences_to_texts(result)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result_text))


## Restore the latest checkpoint and test

In [23]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [24]:
spellcheck(u'aa batteries maintain the settings if the power ever gos off.')

Input: aa batteries maintain the settings if the power ever gos off.
Predicted translation: ['a a   b a t t e r i e s   m a i n t a i n   t h e   s e t t i n g s   i f   t h e   p o w e r   e v e r   g o e s   o f f   . >']


In [25]:
spellcheck(u'aardvark females appear tobe come ointo season once per yaer.')

Input: aardvark females appear tobe come ointo season once per yaer.
Predicted translation: ['a a r d v a r k   f e m a l e s   a p p e a r   t o   b o i n g   o f   c o n t a n   o n   a f r i c a   ,   s o u r t h   o f   t h e   a t r i a l   a n e   p o s t r o b o l   g a n d   . >']


In [26]:
spellcheck(u'mosy aardvarks have lnog snouts.')

Input: mosy aardvarks have lnog snouts.
Predicted translation: ['m o s y   a a r d v a r k s   h a v e   l o n g   s n o u t s   . >']


In [27]:
spellcheck(u'abdominal paine is relieved by defecation.')

Input: abdominal paine is relieved by defecation.
Predicted translation: ['a b d o m i n a l   p a i n   i s   r e l e i v e d   b y   d e f e c a t i o n   . >']


## Use tf-addons BeamSearchDecoder


In [28]:
def beam_evaluate_sentence(sentence, beam_width=3):
  sentence = dataset_creator.sentence_processing(sentence)

  inputs = [inp_lang.word_index[char] for char in sentence]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_input,
                                                          padding='post')
  inputs = tf.convert_to_tensor(inputs)
  inference_batch_size = inputs.shape[0]
  result = ''

  enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
  enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)

  dec_h = enc_h
  dec_c = enc_c

  start_tokens = tf.fill([inference_batch_size], targ_lang.word_index['<'])
  end_token = targ_lang.word_index['>']

  enc_out = tfa.seq2seq.tile_batch(enc_out, multiplier=beam_width)
  decoder.attention_mechanism.setup_memory(enc_out)
  print("beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 16, 1024]] :", enc_out.shape)

  # set decoder_inital_state which is an AttentionWrapperState considering beam_width
  hidden_state = tfa.seq2seq.tile_batch([enc_h, enc_c], multiplier=beam_width)
  decoder_initial_state = decoder.rnn_cell.get_initial_state(batch_size=beam_width*inference_batch_size, dtype=tf.float32)
  decoder_initial_state = decoder_initial_state.clone(cell_state=hidden_state)

  # Instantiate BeamSearchDecoder
  decoder_instance = tfa.seq2seq.BeamSearchDecoder(decoder.rnn_cell,beam_width=beam_width, output_layer=decoder.fc)
  decoder_embedding_matrix = decoder.embedding.variables[0]

  # The BeamSearchDecoder object's call() function takes care of everything.
  outputs, final_state, sequence_lengths = decoder_instance(decoder_embedding_matrix, start_tokens=start_tokens, end_token=end_token, initial_state=decoder_initial_state)


  # Convert the shape of outputs and beam_scores to (inference_batch_size, beam_width, time_step_outputs)
  final_outputs = tf.transpose(outputs.predicted_ids, perm=(0,2,1))
  beam_scores = tf.transpose(outputs.beam_search_decoder_output.scores, perm=(0,2,1))

  return final_outputs.numpy(), beam_scores.numpy()

In [29]:
def beam_spellcheck(sentence):
  result, beam_scores = beam_evaluate_sentence(sentence)
  print(result.shape, beam_scores.shape)
  for beam, score in zip(result, beam_scores):
    print(beam.shape, score.shape)
    output = targ_lang.sequences_to_texts(beam)
    output = [a[:a.index('>')] for a in output]
    beam_score = [a.sum() for a in score]
    print('Input: %s' % (sentence))
    for i in range(len(output)):
      print('{} Predicted Sentence: {}  {}'.format(i+1, output[i], beam_score[i]))


In [30]:
beam_spellcheck(u'mosy aardvarks have lnog snouts.')

beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 16, 1024]] : (3, 109, 1024)
(1, 3, 34) (1, 3, 34)
(3, 34) (3, 34)
Input: mosy aardvarks have lnog snouts.
1 Predicted Sentence: m o s y   a a r d v a r k s   h a v e   l o n g   s n o u t s   .   -11.628411293029785
2 Predicted Sentence: m o s y   a a r d v a r k s   h a v e   l n o g   s n o u t s   .   -173.80020141601562
3 Predicted Sentence: m o s y   a a r d v a r k s   h a v e   l o n h   s n o u t s   .   -201.2129364013672


In [31]:
beam_spellcheck(u'abdominal paine is relieved by defecation.')

beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 16, 1024]] : (3, 109, 1024)
(1, 3, 44) (1, 3, 44)
(3, 44) (3, 44)
Input: abdominal paine is relieved by defecation.
1 Predicted Sentence: a b d o m i n a l   p a i n   i s   r e l e i v e d   b y   d e f e c a t i o n   .   -36.465274810791016
2 Predicted Sentence: a b d o m i n a l   p a i n n   i s   r e l e i v e d   b y   d e f e c a t i o n   .   -136.0968017578125
3 Predicted Sentence: a b d o m i n a l   p a i n e   i s   r e l e i v e d   b y   d e f e c a t i o n   .   -162.90682983398438
